In [9]:
!pip install datasets transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
import numpy as np
import pandas as pd
from scipy import stats

from datasets import load_dataset
from transformers import AutoTokenizer
import torch

### Данные

In [11]:
train_dataset, val_dataset, test_dataset = load_dataset(
    'cnn_dailymail',
    split=['train', 'validation', 'test'],
    name='3.0.0'
)

  0%|          | 0/3 [00:00<?, ?it/s]

### Токенайзер

In [12]:
model_name = "facebook/bart-base"
tokenizer_name = model_name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

### Длины (в токенах) статей val датасета

In [13]:
def get_tokens_len(examples, column="article"):
    inputs = examples[column]
    model_inputs = tokenizer(inputs)
    model_inputs_len = [len(model_input) for model_input in model_inputs['input_ids']]
    return model_inputs_len

In [21]:
val_dataset_tokenized_lens = np.array(get_tokens_len(val_dataset))

In [20]:
val_dataset_tokenized_lens.shape[0]

13368

In [22]:
val_dataset_tokenized_lens.mean()

847.5531119090365

In [24]:
np.quantile(val_dataset_tokenized_lens, 0.75)

1087.25

In [26]:
stats.percentileofscore(val_dataset_tokenized_lens, 1024)

71.20362058647517

### Длины summary в валидационном датасете

In [27]:
val_dataset_labels_tokenized_lens = np.array(
    get_tokens_len(val_dataset, 'highlights')
)

In [28]:
val_dataset_labels_tokenized_lens.mean()

75.71476660682227

In [29]:
val_dataset_labels_tokenized_lens.max()

2020

In [32]:
stats.percentileofscore(val_dataset_labels_tokenized_lens, 256)

99.95511669658887

In [33]:
stats.percentileofscore(val_dataset_labels_tokenized_lens, 128)

94.96932974266906